### Libraries import

In [1]:
from dotenv import load_dotenv
import os
import requests
import urllib3
import pandas as pd
from tqdm import tqdm
import time
import random
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
pd.set_option('display.max_columns', None)

In [2]:
from sqlalchemy import Text, Integer, BigInteger, Float, Boolean
from sqlalchemy import create_engine
from sqlalchemy.dialects.postgresql import JSONB

### Load evn variables

In [3]:
load_dotenv()

True

### Strava API setup

In [4]:
auth_url = 'https://www.strava.com/oauth/token'
activites_url = 'https://www.strava.com/api/v3/athlete/activities'

payload = {
    'client_id': os.getenv('CLIENT_ID'),
    'client_secret': os.getenv('CLIENT_SECRET'),
    'refresh_token': os.getenv('REFRESH_TOKEN'),
    'grant_type': 'refresh_token',
    'f': 'json'
}

### SQL setup

In [5]:
engine = create_engine(os.getenv('DB_URI'))

### Requesing list of all activities

In [6]:
print('Requesting Token...\n')
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']

print('Access Token = {}\n'.format(access_token))
header = {'Authorization': 'Bearer ' + access_token}

# The first loop, request_page_number will be set to one, so it requests the first page. Increment this number after
# each request, so the next time we request the second page, then third, and so on...
request_page_num = 1
all_activities = []

while True:
    param = {'per_page': 200, 'page': request_page_num}
    # initial request, where we request the first page of activities
    my_dataset = requests.get(activites_url, headers=header, params=param).json()

    if my_dataset.get('message') == 'Rate Limit Exceeded':
        print('breaking out of while loop because rate limit was exceeded')
        raise SystemExit("Stop here")

    # check the response to make sure it is not empty. If it is empty, that means there is no more data left. So if you have
    # 1000 activities, on the 6th request, where we request page 6, there would be no more data left, so we will break out of the loop
    if len(my_dataset) == 0:
        print('breaking out of while loop because the response is zero, which means there must be no more activities')
        break

    # if the all_activities list is already populated, that means we want to add additional data to it via extend.
    if all_activities:
        print('all_activities is populated')
        all_activities.extend(my_dataset)

    # if the all_activities is empty, this is the first time adding data so we just set it equal to my_dataset
    else:
        print('all_activities is NOT populated')
        all_activities = my_dataset

    request_page_num += 1

print(len(all_activities))
for count, activity in enumerate(all_activities):
    print(activity['name'])
    print(count)

all_activities_df = pd.json_normalize(all_activities, sep='_')

Requesting Token...

Access Token = aa35c3d7104d94c3bb32642bda61a783237916b3

breaking out of while loop because rate limit was exceeded


SystemExit: Stop here

/opt/anaconda3/envs/strava-env/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Check which details are missing

In [7]:
activities_id_db_query = "SELECT id FROM bronze.activities_details"
activities_id_db_df = pd.read_sql(activities_id_db_query, engine)

In [8]:
all_activities_df['id']

0       15786538213
1       15775473094
2       15766076458
3       15760949041
4       15729456618
           ...     
1084     6012647108
1085     6012647093
1086     6012647092
1087     6012647083
1088     6012647095
Name: id, Length: 1089, dtype: int64

In [9]:
activities_id_db_df['id']

0      15786538213
1      14115127012
2      14086094444
3      14080030310
4      12956260994
          ...     
169     6012647108
170     6012647093
171     6012647092
172     6012647083
173     6012647095
Name: id, Length: 174, dtype: int64

In [10]:
missing_rows_df = all_activities_df[~all_activities_df['id'].isin(activities_id_db_df['id'])]
missing_ids = missing_rows_df['id'].unique()
missing_ids_df = pd.DataFrame({'id': missing_ids})
missing_ids_df

,id
0,15775473094
1,15766076458
2,15760949041
3,15729456618
4,15716821076
...,...
910,6316153411
911,6312219800
912,6298683669
913,6293877210


### Requesing list of all activities with details

In [11]:
print('Requesting Token...\n')
res = requests.post(auth_url, data=payload, verify=False) # auth_url & payload referenced in code already above
access_token = res.json()['access_token']
header = {'Authorization': 'Bearer ' + access_token}
print('Access Token = {}\n'.format(access_token))

all_activities_details = []
count = 0

print('Calculating time to get all activities details....\n')
print(missing_ids_df.shape[0] * 8 / 60, ' minutes to obtain data')

print('Getting details of each activity')
for index, row in tqdm(missing_ids_df.iterrows(), total=missing_ids_df.shape[0]):
    get_activity_url = 'https://www.strava.com/api/v3/activities/{}'.format(row['id'])
    try:
        activity_details = requests.get(get_activity_url, headers=header).json() 
        all_activities_details.append(activity_details)
        time.sleep(random.randint(7, 9))
    except requests.exception.RequestException as e:
        raise SystemExit(e)

all_activities_details_df = pd.json_normalize(all_activities_details, sep='_')

Requesting Token...

Access Token = aa35c3d7104d94c3bb32642bda61a783237916b3

Calculating time to get all activities details....

122.0  minutes to obtain data
Getting details of each activity


100%|██████████| 915/915 [2:22:49<00:00,  9.37s/it]   


In [21]:
all_activities_details_df = pd.json_normalize(all_activities_details, sep='_')

In [ ]:
all_activities_details_df = all_activities_details_df.drop(['message', 'errors'], axis=1, errors='ignore')

In [ ]:
all_activities_details_df = all_activities_details_df.dropna(how='all')

In [26]:
all_activities_details_df

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,description,calories,perceived_exertion,prefer_perceived_exertion,segment_efforts,laps,stats_visibility,hide_from_home,device_name,embed_token,available_zones,athlete_id,athlete_resource_state,map_id,map_polyline,map_resource_state,map_summary_polyline,photos_primary,photos_count,workout_type,average_cadence,average_watts,max_watts,weighted_average_watts,device_watts,kilojoules,splits_metric,splits_standard,best_efforts,gear_primary,gear_name,gear_nickname,gear_resource_state,gear_retired,gear_distance,gear_converted_distance,similar_activities_effort_count,similar_activities_average_speed,similar_activities_min_average_speed,similar_activities_mid_average_speed,similar_activities_max_average_speed,similar_activities_pr_rank,similar_activities_frequency_milestone,similar_activities_trend_speeds,similar_activities_trend_current_activity_index,similar_activities_trend_min_speed,similar_activities_trend_mid_speed,similar_activities_trend_max_speed,similar_activities_trend_direction,similar_activities_resource_state,average_temp,photos_primary_unique_id,photos_primary_urls_600,photos_primary_urls_100,photos_primary_source,photos_primary_media_type,photos_use_primary_photo,private_note
0,3.0,Afternoon Weight Training,0.0,3576.0,3576.0,0.0,Workout,WeightTraining,1.577547e+10,2025-09-11T13:01:52Z,2025-09-11T15:01:52Z,(GMT+02:00) Africa/Blantyre,7200.0,NaN,NaN,NaN,0.0,8.0,2.0,1.0,0.0,True,False,False,False,followers_only,False,None,[],[],0.000,0.000,True,97.5,140.0,False,True,0.0,0.0,1.685060e+10,16850602738,garmin_ping_479160572044,False,0.0,0.0,False,8.0,"Reska8️⃣7️⃣\nSumo deadlift PR: 147,5kg🗿",291.0,NaN,None,[],"[{'id': 56177598146, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin Forerunner 970,30dcb8d21721ccb941fded27b51f710e5eb81df3,[heartrate],81055898.0,1.0,a15775473094,,3.0,,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,7.5km Easy Run😌,7556.8,2651.0,2651.0,11.0,Run,Run,1.576608e+10,2025-09-10T15:44:58Z,2025-09-10T17:44:58Z,(GMT+01:00) Europe/Warsaw,7200.0,NaN,NaN,NaN,0.0,9.0,0.0,1.0,0.0,False,False,False,False,everyone,False,g24134620,"[51.107271, 17.124]","[51.103853, 17.120931]",2.851,3.580,True,144.3,156.0,False,True,124.8,116.2,1.684065e+10,16840645258,garmin_ping_478851416260,False,0.0,0.0,False,30.0,7.5km Easy Run with Runna ✅\n\nLuźne klepanie🫡...,582.0,NaN,None,"[{'id': 3400905546860626570, 'resource_state':...","[{'id': 56144065886, 'resource_state': 2, 'nam...","[{'type': 'heart_rate', 'visibility': 'everyon...",False,Garmin Forerunner 970,2814aff65652a4fbab20db29c347c5a308e79cc8,"[heartrate, pace, power]",81055898.0,1.0,a15766076458,m{|vH}oogBHRD\Vb@Ph@HPLBZa@LINHXb@HBFATSVc@l@e...,3.0,k_|vHcpogB^v@b@dAJNNHT@n@Q\GTARBRHPRd@r@FRBv@J...,NaN,0.0,0.0,83.8,349.9,456.0,348.0,True,927.8,"[{'distance': 1000.9, 'elapsed_time': 353, 'el...","[{'distance': 1610.0, 'elapsed_time': 561, 'el...","[{'id': 66139861602, 'resource_state': 2, 'nam...",False,ASICS Novablast 5,None,2.0,False,281932.0,281.9,19.0,2.688259,1.858722,2.296345,3.003496,NaN,NaN,"[2.7006108076039785, 2.7660581928959, 2.893624...",4.0,1.858722,2.296345,3.003496,-1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Morning Weight Training,0.0,3625.0,3625.0,0.0,Workout,WeightTraining,1.576095e+10,2025-09-10T07:01:39Z,2025-09-10T09

### Requesing kudos related to activities

In [14]:
# print('Requesting Token...\n')
# res = requests.post(auth_url, data=payload, verify=False)
# access_token = res.json()['access_token']
# header = {'Authorization': 'Bearer ' + access_token}
# print('Access Token = {}\n'.format(access_token))

# all_activities_kudos = []
# count = 0

# print('Calculating time to get all activities details....\n')
# print(all_activities_df.shape[0] * 8 / 60, ' minutes to obtain data')

# print('Getting details of each activity')
# for index, row in tqdm(all_activities_df.iterrows(), total=all_activities_df.shape[0]):
#     get_kudos_url = 'https://www.strava.com/api/v3/activities/{}/kudos'.format(row['id'])
#     try:
#         activity_kudos = requests.get(get_kudos_url, headers=header).json()
#         if len(activity_kudos) > 0:
#             for kudos in activity_kudos:
#                 kudos['activity_id'] = row['id']
#                 all_activities_kudos.extend(activity_kudos)
#         time.sleep(random.randint(7, 9))
#     except requests.exception.RequestException as e:
#         raise SystemExit(e)
    
# all_activities_kudos_df = pd.json_normalize(all_activities_kudos, sep='_')

In [ ]:
# all_activities_kudos_df

### Load data to PostgreSQL

In [27]:
with engine.begin() as conn:
    conn.exec_driver_sql("CREATE SCHEMA IF NOT EXISTS bronze;")

activities_dtype_map = {
    "resource_state": Integer,
    "name": Text,
    "distance": Float,
    "moving_time": Integer,
    "elapsed_time": Integer,
    "total_elevation_gain": Float,
    "type": Text,
    "sport_type": Text,
    "workout_type": Float,
    "id": BigInteger,
    "start_date": Text,
    "start_date_local": Text,
    "timezone": Text,
    "utc_offset": Float,
    "location_city": Text,
    "location_state": Text,
    "location_country": Text,
    "achievement_count": Integer,
    "kudos_count": Integer,
    "comment_count": Integer,
    "athlete_count": Integer,
    "photo_count": Integer,
    "trainer": Boolean,
    "commute": Boolean,
    "manual": Boolean,
    "private": Boolean,
    "visibility": Text,
    "flagged": Boolean,
    "gear_id": Text,
    "start_latlng": JSONB,
    "end_latlng": JSONB,
    "average_speed": Float,
    "max_speed": Float,
    "average_cadence": Float,
    "average_watts": Float,
    "max_watts": Float,
    "weighted_average_watts": Float,
    "device_watts": Boolean,
    "kilojoules": Float,
    "has_heartrate": Boolean,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "heartrate_opt_out": Boolean,
    "display_hide_heartrate_option": Boolean,
    "elev_high": Float,
    "elev_low": Float,
    "upload_id": BigInteger,
    "upload_id_str": Text,
    "external_id": Text,
    "from_accepted_tag": Boolean,
    "pr_count": Integer,
    "total_photo_count": Integer,
    "has_kudoed": Boolean,
    "suffer_score": Float,
    "athlete_id": BigInteger,
    "athlete_resource_state": Integer,
    "map_id": Text,
    "map_summary_polyline": Text,
    "map_resource_state": Integer,
    "average_temp": Float,
}

activities_details_dtype_map = {
    "resource_state": Integer,
    "name": Text,
    "distance": Float,
    "moving_time": Integer,
    "elapsed_time": Integer,
    "total_elevation_gain": Float,
    "type": Text,
    "sport_type": Text,
    "workout_type": Float,
    "id": BigInteger,
    "start_date": Text,
    "start_date_local": Text,
    "timezone": Text,
    "utc_offset": Float,
    "location_city": Text,
    "location_state": Text,
    "location_country": Text,
    "achievement_count": Integer,
    "kudos_count": Integer,
    "comment_count": Integer,
    "athlete_count": Integer,
    "photo_count": Integer,
    "trainer": Boolean,
    "commute": Boolean,
    "manual": Boolean,
    "private": Boolean,
    "visibility": Text,
    "flagged": Boolean,
    "gear_id": Text,
    "start_latlng": JSONB,
    "end_latlng": JSONB,
    "average_speed": Float,
    "max_speed": Float,
    "average_cadence": Float,
    "average_watts": Float,
    "max_watts": Float,
    "weighted_average_watts": Float,
    "device_watts": Boolean,
    "kilojoules": Float,
    "has_heartrate": Boolean,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "heartrate_opt_out": Boolean,
    "display_hide_heartrate_option": Boolean,
    "elev_high": Float,
    "elev_low": Float,
    "upload_id": BigInteger,
    "upload_id_str": Text,
    "external_id": Text,
    "from_accepted_tag": Boolean,
    "pr_count": Integer,
    "total_photo_count": Integer,
    "has_kudoed": Boolean,
    "suffer_score": Float,
    "description": Text,
    "calories": Float,
    "perceived_exertion": Text,
    "prefer_perceived_exertion": Text,
    "segment_efforts": JSONB,
    "splits_metric": JSONB,
    "splits_standard": JSONB,
    "laps": JSONB,
    "best_efforts": JSONB,
    "stats_visibility": JSONB,
    "hide_from_home": Boolean,
    "device_name": Text,
    "embed_token": Text,
    "available_zones": JSONB,
    "athlete_id": BigInteger,
    "athlete_resource_state": Integer,
    "map_id": Text,
    "map_polyline": Text,
    "map_resource_state": Integer,
    "map_summary_polyline": Text,
    "gear_primary": Boolean,
    "gear_name": Text,
    "gear_nickname": Text,
    "gear_resource_state": Float,
    "gear_retired": Boolean,
    "gear_distance": Float,
    "gear_converted_distance": Float,
    "photos_primary": JSONB,
    "photos_count": Integer,
    "similar_activities_effort_count": Float,
    "similar_activities_average_speed": Float,
    "similar_activities_min_average_speed": Float,
    "similar_activities_mid_average_speed": Float,
    "similar_activities_max_average_speed": Float,
    "similar_activities_pr_rank": Float,
    "similar_activities_frequency_milestone": Float,
    "similar_activities_trend_speeds": JSONB,
    "similar_activities_trend_current_activity_index": Float,
    "similar_activities_trend_min_speed": Float,
    "similar_activities_trend_mid_speed": Float,
    "similar_activities_trend_max_speed": Float,
    "similar_activities_trend_direction": Float,
    "similar_activities_resource_state": Float,
    "average_temp": Float,
    "photos_primary_unique_id": Text,
    "photos_primary_urls_600": Text,
    "photos_primary_urls_100": Text,
    "photos_primary_source": Integer,
    "photos_primary_media_type": Integer,
    "photos_use_primary_photo": Boolean,
    "private_note": Text,
    "message": Text,
    "errors": JSONB
}

# activities_kudos_dtype_map = {
#     "resource_state": Integer,
#     "firstname": Text,
#     "lastname": Text,
#     "activity_id": BigInteger,
# }


all_activities_df.to_sql(
    name="activities",
    schema="bronze",
    con=engine,
    if_exists="replace",
    index=False,
    dtype=activities_dtype_map,
    method=None,
    chunksize=1000
)

all_activities_details_df.to_sql(
    name="activities_details",
    schema="bronze",
    con=engine,
    if_exists="append",
    index=False,
    dtype=activities_details_dtype_map,
    method=None,
    chunksize=1000
)

# all_activities_kudos_df.to_sql(
#     name="kudos",
#     schema="bronze",
#     con=engine,
#     if_exists="append",
#     index=False,
#     dtype=activities_kudos_dtype_map,
#     method="multi",
#     chunksize=1000
# )

-1

In [28]:
len(activities_details_dtype_map)

107

In [29]:
len(all_activities_details_df.columns)

105